In [1]:
# estimate or drop missing features
# select a date range?
# transform categorical features into str labels

# pre-processing -- 
# define outcome
# earliest intn shipping - 1331091810  -- restrict to this range
# set unspecified to 0
# drop 0 launch times, -1

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math
from IPython.display import display
from IPython.core.pylabtools import getfigs
from sklearn import linear_model 
import time
import pymysql as mdb
import re
#import seaborn as sns

In [3]:
%matplotlib qt

In [4]:
from prep_predictors_for_ml import *
%load_ext autoreload
%autoreload 2

In [5]:
# in sql query, specify: 
# launch_t > 0, 
# n_rewards > -1, 
# ship_date > -1
# launch t >= 1331091810 (March 6, 2012 - earliest intn ship date)
# status != live

In [6]:
# run sql query, convert to df

In [7]:
predictor_df = pd.read_pickle('predictor_df_unprocessed.save')

In [8]:
predictor_df.columns

Index([u'Id', u'Url', u'Outcome', u'Pledged', u'Goal', u'Category',
       u'Subcategory', u'Currency', u'Country', u'Location', u'Launch_s',
       u'Campaign_days', u'Title_n_words', u'Blurb_n_words', u'Month',
       u'Monday_day', u'Week_day', u'Hour', u'N_rewards', u'Min_reward',
       u'Median_reward', u'Max_reward', u'Reward_ship_days', u'Ships_intn',
       u'N_links', u'N_figs', u'Body_n_words', u'N_creator_backed'],
      dtype='object')

In [12]:
predictor_df.tail()

,Id,Url,Outcome,Pledged,Goal,Category,Subcategory,Currency,Country,Location,...,N_rewards,Min_reward,Median_reward,Max_reward,Reward_ship_days,Ships_intn,N_links,N_figs,Body_n_words,N_creator_backed
133066,191380,https://www.kickstarter.com/projects/207940292...,suspended,0,2500,Installations,art/installations,USD,US,prior-lake-mn,...,1,25,25,25,92,-1,0,0,28,0
133067,191381,https://www.kickstarter.com/projects/184521459...,suspended,420,2000,Mixed Media,art/mixed media,USD,US,memphis-tn,...,14,5,40,150,41,1,18,13,750,0
133068,191382,https://www.kickstarter.com/projects/209708895...,suspended,0,31,Painting,art/painting,USD,US,lakeland-fl,...,3,8,31,1001,111,0,2,0,504,0
133069,191383,https://www.kickstarter.com/projects/thelifecu...,suspended,10084,15000,Public Art,art/public art,USD,US,black-rock-city-nv,...,5,10,200,1000,155,1,1,7,1246,130
133070,191384,https://www.kickstarter.com/projects/reaper889...,suspended,300,300000,Painting,art/painting,USD,US,douglasville-douglas-ga,...,4,300,750,5000,136,0,0,0,624,0


In [9]:
# update this with new features 
X_cols = ['Goal','Campaign_days','Title_n_words','Blurb_n_words',
          'Hour','N_rewards','Min_reward','Median_reward','Max_reward',
          'Reward_ship_days','N_links','N_figs','Body_n_words','N_creator_backed']
# Categorical features to add 
# project category
# month
# month day
# weekday
# ships internationally

In [10]:
subset_predictor_df = predictor_df.ix[:,X_cols]

In [11]:
subset_predictor_df.head(3)

,Goal,Campaign_days,Title_n_words,Blurb_n_words,Hour,N_rewards,Min_reward,Median_reward,Max_reward,Reward_ship_days,N_links,N_figs,Body_n_words,N_creator_backed
0,5000,30,2,17,20,8,10,175,5000,110,1,0,812,18
1,10000,30,8,21,14,10,5,125,5000,31,5,1,792,7
2,18000,27,5,14,8,19,5,500,10000,4,0,0,540,3


In [13]:
outcomes_df = make_success_fail_df(predictor_df.ix[:,['Pledged','Goal']])
outcomes_df.head(2)

,Outcome
0,1
1,1


In [14]:
new_ships_intn_df = make_ships_intn_df(predictor_df)
new_ships_intn_df.head(2)

,Ships_intn
0,0
1,0


In [15]:
def shorten_category(subcategory):
    match = re.search('(\w*)(\/*)',subcategory)
    if match:
        return match.group(1)

In [16]:
n_rows = np.shape(predictor_df)[0]
print n_rows

133071


In [19]:
# make ks_categories until I fix this in my new predictor df
ks_categories_df = pd.DataFrame(index=range(n_rows),columns=['Category'],dtype='str')
# now fix my category features
for i in range(n_rows): 
    ks_categories_df.ix[i,'Category'] = shorten_category(predictor_df.ix[i,'Subcategory'])

In [24]:
categories_dummy_df = make_categorical_df_from_str(ks_categories_df)
categories_dummy_df.head(2)

,art,comics,crafts,dance,design,fashion,film,food,games,journalism,music,photography,publishing,technology,theater
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [46]:
weekday_df = make_categorical_df_from_num(predictor_df.loc[:,['Week_day']],\
                 col_names=['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])
weekday_df.tail(2)

,Mon,Tue,Wed,Thu,Fri,Sat,Sun
133069,0,0,0,1,0,0,0
133070,0,0,0,0,1,0,0


In [44]:
hour_df = make_categorical_df_from_num(predictor_df.loc[:,['Hour']],append_str='h')
hour_df.tail(3)

,0h,1h,2h,3h,4h,5h,6h,7h,8h,9h,...,14h,15h,16h,17h,18h,19h,20h,21h,22h,23h
133068,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
133069,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
133070,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
month_day_df = make_categorical_df_from_num(predictor_df.loc[:,['Monday_day']],col_names=range(1,32),append_str='d') # fix this misspelling
month_day_df.tail(2)

,1d,2d,3d,4d,5d,6d,7d,8d,9d,10d,...,22d,23d,24d,25d,26d,27d,28d,29d,30d,31d
133069,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
133070,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [67]:
month_df = make_categorical_df_from_num(predictor_df.loc[:,['Month']],\
         col_names=['Jan','Feb','Mar','Apr','May','Jun','Jul',\
                    'Aug','Sep','Oct','Nov','Dec'])
month_df.tail(2)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
133069,0,0,0,0,1,0,0,0,0,0,0,0
133070,0,0,0,0,0,0,0,0,0,1,0,0


In [82]:
X_df = pd.concat([subset_predictor_df,categories_dummy_df, new_ships_intn_df,month_df, \
                                   weekday_df, month_day_df, hour_df],axis=1)

In [83]:
X_df.head(2)

,Goal,Campaign_days,Title_n_words,Blurb_n_words,Hour,N_rewards,Min_reward,Median_reward,Max_reward,Reward_ship_days,...,14h,15h,16h,17h,18h,19h,20h,21h,22h,23h
0,5000,30,2,17,20,8,10,175,5000,110,...,0,0,0,0,0,0,1,0,0,0
1,10000,30,8,21,14,10,5,125,5000,31,...,1,0,0,0,0,0,0,0,0,0


In [84]:
X_df.to_pickle('X_df.save')

In [52]:
outcomes_df.to_pickle('outcomes_df.save')